Demonstrate Sentinel-2 data access using PySTAC. This notebook derives heavily from the [planetary computer example here](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook)

In [1]:
# cloud/STAC related imports
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
import planetary_computer as pc

# GIS imports
import rasterio
from rasterio import windows, features, warp
import geopandas as gpd
import json
import rasterio.mask

# misc imports
from pathlib import Path
from typing import List

In [2]:
# AoI to query
shapefile = Path('../data/shapefiles/amazon_river/aoi.shp')
df = gpd.read_file(shapefile)
shape_geojson = json.loads(df.to_json())['features'][0]['geometry']

# ToI to query
time_of_interest = "2019-06-01/2019-08-01"

In [3]:
# Query the planetary computer and print # of results returned 
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=shape_geojson,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}}, # percentage acceptable cloud cover
)

# Check how many items were returned
items = list(search.get_items())
print(f"Returned {len(items)} Items")

Returned 8 Items


In [4]:
# Sort by cloud coverage and pick image with least cloud cover
least_cloudy_item = sorted(items, key=lambda item: eo.ext(item).cloud_cover)[0]

print(
    f"Choosing {least_cloudy_item.id} from {least_cloudy_item.datetime.date()}"
    f" with {eo.ext(least_cloudy_item).cloud_cover}% cloud cover"
)

Choosing S2A_MSIL2A_20190729T141051_R110_T21MXU_20201106T050220 from 2019-07-29 with 0.32129% cloud cover


In [5]:
# Obtain href which will serve up the image
asset_href = least_cloudy_item.assets["visual"].href
signed_href = pc.sign(asset_href)

In [6]:
# Windowed read of the asset
with rasterio.open(signed_href) as ds:
    aoi_bounds = features.bounds(shape_geojson) # note that aoi_bounds can exceed area defined by shape
    warped_aoi_bounds = warp.transform_bounds("epsg:4326", ds.crs, *aoi_bounds)
    aoi_window = windows.from_bounds(transform=ds.transform, *warped_aoi_bounds)
    band_data = ds.read(window=aoi_window)
    profile = ds.profile
    img_bounds = ds.bounds

In [7]:
def return_overlap_bounds(*bounds_list):
    return [max(_b) if i<2 else min(_b) for i, _b in enumerate(zip(*bounds_list))]
    
overlap_bounds = return_overlap_bounds(img_bounds, warped_aoi_bounds)

In [8]:
# Obtain Affine transform corresponding to window using overlap bounds and image size
overlap_transform = rasterio.transform.from_bounds(*overlap_bounds, band_data.shape[2], band_data.shape[1])

In [9]:
# Modify 
band_profile = profile.copy()
band_profile['transform'] = overlap_transform
band_profile['height'] = band_data.shape[1]
band_profile['width'] = band_data.shape[2]

In [10]:
with rasterio.open(f"../data/{asset_href.split('/')[-1]}", 'w', **band_profile) as ds:
    ds.write(band_data)